In [2]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent , AgentExecutor
import requests
from datetime import datetime
import os
import platform

In [3]:
llm = ChatOllama(model="llama3.2:3b")

In [4]:
# define tools
@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia for a given query and return the page title."""
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{query}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('title', 'No title found')
    else:
        return "Could not search for this query"

@tool
def get_current_time() -> str:
    """Return the current date and time in 'YYYY-MM-DD HH:MM:SS' format."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def calculate_math(expression: str) -> str:
    """Evaluate a basic math expression and return the result as a string."""
    try:
        result = eval(expression)
        return str(result)
    except Exception:
        return "Invalid math expression"
    
@tool
def restart_my_pc() -> str:
    """Restart this computer. Only use this in a safe and trusted environment."""
    try:
        system = platform.system()
        if system == "Windows":
            os.system("shutdown /r /t 0")
        elif system == "Linux" or system == "Darwin":  # Darwin is macOS
            os.system("sudo reboot")
        else:
            return f"Unsupported operating system: {system}"
        return "Restart command executed."
    except Exception as e:
        return f"Failed to restart: {str(e)}"
    

In [5]:
tools = [search_wikipedia,get_current_time,calculate_math,restart_my_pc]

In [6]:
prompt = ChatPromptTemplate([
    ("system","""
     you are a helpful research assistant,you have a access to tools that can help you answer questions.
     Avilable tools:
     - search_wikipedia : Search Wikipedia for a given query and return the page title.
     - get_current_time : Return the current date and time in 'YYYY-MM-DD HH:MM:SS' format.
     - calculate_math : Evaluate a basic math expression and return the result as a string.
     - restart_my_pc : Restart this computer. Only use this in a safe and trusted environment.

     use tools when needed to provide accurate responses
     """),
     MessagesPlaceholder(variable_name="chat_history"),
     ("human","{input}"),
     MessagesPlaceholder(variable_name="agent_scratchpad"),

])

In [7]:
agent = create_tool_calling_agent(llm,tools,prompt)
agent_executer = AgentExecutor(agent=agent, tools=tools,verbose=True) 


In [8]:
def research_agent_chat(query):
    res = agent_executer.invoke({
        "input": query,
        "chat_history":[]

    })
    return res["output"]

In [31]:
research_agent_chat("what time is is?")



> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{}`


2025-06-30 12:49:21The current time is 12:49:21 on June 30, 2025.

> Finished chain.


'The current time is 12:49:21 on June 30, 2025.'

In [32]:
research_agent_chat("what is AI ?")



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Artificial Intelligence'}`


Artificial intelligenceArtificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. The term "artificial" means that the system is created by humans, while "intelligence" implies that it possesses abilities similar to those of a human mind.

The main goal of AI is to create machines that can think and act like humans, and to make them capable of performing tasks that would typically require human intelligence. This includes applications such as:

*   Virtual assistants: AI-powered virtual assistants, like Siri, Alexa, and Google Assistant, can understand natural language and perform tasks such as setting reminders, sending messages, and making calls.
*   Image recognition: AI algorithms can be trained to recognize patterns in images, allo

'Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. The term "artificial" means that the system is created by humans, while "intelligence" implies that it possesses abilities similar to those of a human mind.\n\nThe main goal of AI is to create machines that can think and act like humans, and to make them capable of performing tasks that would typically require human intelligence. This includes applications such as:\n\n*   Virtual assistants: AI-powered virtual assistants, like Siri, Alexa, and Google Assistant, can understand natural language and perform tasks such as setting reminders, sending messages, and making calls.\n*   Image recognition: AI algorithms can be trained to recognize patterns in images, allowing them to identify objects, people, and scenes with great accuracy.\n*   Natural Language Processing (NLP): AI-powered NLP systems

In [9]:
research_agent_chat("calculate 10+5*2")



> Entering new AgentExecutor chain...

Invoking: `calculate_math` with `{'expression': '10+5*2'}`


20The result of the calculation `10+5*2` is 20.

> Finished chain.


'The result of the calculation `10+5*2` is 20.'

In [ ]:
research_agent_chat("restart my computer")



> Entering new AgentExecutor chain...
